# Problem 3: High dimensional linear system 

This experiment test how well the global and stepwise algorithms handle high-dimensional data. 
     
### MDP

- States: 30 dimensional state vector 
    - $X_t = [X_{1t}, ... ,X_{15t}]^T$ , 15 exogenous state variables 
    - $E_t = [E_{1t}, ... ,E_{15t}]^T$ , 15 endogenous state variables 
        
- State transition function: 
    - $X_{t+1} = M_x * X_t + ϵ_x$
    - $E_{t+1} = M_e * [E_t, X_t, A_t]^T + ϵ_e$
    - where $M_x$ & $M_e$ are the transition functions for the exogenous MRP & endogenous MDP, generated according to $N(0,1)$
    - $ϵ_x$ is the exogenous normal noise distribution $N(0, 0.09)$ and 
    - $ϵ_e$ is the endogenous normal noise distirbution $N(0, 0.04)$ 

- $s_0$: the initial state is a zero vector 

- The observed state vector is a linear mixture of the hidden exogenous & endogenous states defined as :
    - $S_t = M * [E_t, X_t]^T$ where $M$ is $30x30$ element of the reals generated according to $N(0,1)$

- Reward: $R_t = R_{xt} + R_{et}$ where $R_{xt}$ is the endogenous reward & $R_{et}$ is the exogenous reward 


### Experiments 

Neural Net: 
- input: 30 dimensional state vector of endogenous & exogenous components
- layer(s): a single hidden layer of 20 tanh units 
- output: linear q function for each value of the current state, $s_t$  

All 4 Q-learners: 
- observe the entire current state $s_t$
- are initialized identically and employ the same random seed 
- ɣ = 0.9, the discount factor
- α = 0.05, the learning rate

Difference between Q-learners:
- The "full Q-learner" is trained on the full reward  
- The endogenous reward Q-learners are trained on the (estimated) endogenous reward 
- For the first $L$ steps, where $L=1000$ steps, the full reward is employed, & we collect a database of $(s,a,s,r')$ transitions 
- After these $L$ steps, we apply the Global & Stepwise algorithms to estimate $W_x$ & $W_e$
- Then the algorithms fit a linear regression model $R_{exo}(W_x^Ts)$ to predict the reward $r$ as a function of the exogenous state $x=W_x^Ts$

Q-function Use: the Q function is used for action selection 
- `softmax equation' - from NN output: 
$a_t \sim π(s, a_t) = \frac{\exp(Q(s_t, a)/β)}{\sum\limits_{i}\exp(Q(s_t, a)/β)}$

- β = 1.0, the temperature for Boltzmann exploration
- Used steepest descent solving in Manopt 
- The PCC constraint epsilon is 0.05

## Q-learning w/ Softmax Action Selection using Global & Stepwise Algorithms to Identify Exogenous State Variables & Rewards (procedural form):

### PART 0: Initialize Values

Initialize:
   - Q values randomly
   - temperature: beta = 1.0
   - learning rate: alpha = 0.05
   - discount: gamma = 0.9 (for all Q-learners)
   - first state s0 as zero 
   - first L steps where full reward is used for all 4 Q-learners: L = 1000
   - time-step for graph: T=1


### PART 1: Collect transitions to predict the exogenous reward, R_exo

__This is Q-learning with softmax action selection and the added detail of decomposing the reward into exogenous & endogenous components.__

For each time-step from 0 to L: 
- Choose action from current state, a_t & s_t, using softmax policy, pi(s_t, a_t), derived from Q-values: 
```
a_t = exp[Q(s_t, a_t)/temperature] / exp[Q(s_t,a_i)/temperature]
```  
- Store s_t & a_t in transition database
- Take action, a_t, observe r, s_t+1, where r is computed as full reward 

```
Compute the full reward: R_t = R_x,t + R_e,t
    
- R_exo,t = -3 * avg(X_t)    where X_t is the exogenous state transition
- R_end,t = exp[-|avg(E_t)-1|]   where E_t is the endogenous state transition
```
- Store r_t & s_t+1 in transition database
- Update Q-value: 
```
Q(s_t, a_t) <-- Q(s_t, a_t) + alpha * [r + gamma * max_a (Q(s_t+1, a_t+1) - Q(s_t, a_t))]
```
    
- Update current state as next state: 
```
s <-- s'
```

### PART 2: Apply Algorithm 1 & 2 to the transition database to estimate W_x & W_e

- Use database of full reward collected transitions (s,a,s',r) to compute estimate of projected next state of W_x & W_e 


- Compute exogenous reward: fit a linear regression model using estimate of W_x & W_e (we do this to predict the reward, r, as a function of the exogenous state, x = W_x^T * s in Part 3). Compute exogenous reward as: 
```
R_exo = (W_x^T * s) 
```



- Compute endogenous reward: as residual of linear regression model
```
R_end(s) = r - R_exo(x)
```

### PART 3: Endogenous Q-learner (global, stepwise & oracle) uses computed R_end for steps L to 5000  
    
For each time-step from L to 5000: 

- Choose action from current state, a_t & s_t, using softmax policy, pi(s_t, a_t), derived from Q-values: 
```
a_t = exp[Q(s_t, a_t)/temperature] / exp[Q(s_t,a_i)/temperature]
```    
    
- Take action, a_t, observe endogenous r_end, s_t+1, where r_end is as computed in PART 2??? 
    
- Update Q-value: 
```
Q(s_t, a_t) <-- Q(s_t, a_t) + alpha * [r + gamma * max_a (Q(s_t+1, a_t+1) - Q(s_t, a_t))]
```
    
- Update current state as next state: 
```
s <-- s'
```
   
Loop until 5000 actions have been executed 

In [1]:
import numpy as np

### MDP

__States:__ 30 dimensional, with exogenous & endogenous components

In [2]:
# Initial state is zero vector 
states = np.zeros((1,30))
print(states.dtype)
print(states)

float64
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]]


__State Transitions:__ 

Implement the following:
    - transition functions for the exo MRP & endo MDP generated according to N(0,1)
        - M_x an element of the reals with dimensionality 15x15
        - M_e an element of the reals with dimensionality 15x31
        - where each row of each matrix is normalized to sum to 0.99 for stability  
    - gaussian noise distribution variables
        - Epsilon_x generated by N(0,0.09)
        - Epsilon_e generated by N(0,0.04)

In [3]:
# functions to verify that l2-norm is working
'''
def sum_sqrt(a):
    return np.sqrt(np.sum(np.abs(a)**2, axis=-1))

def apply_norm_along_axis(a):
    return np.apply_along_axis(np.linalg.norm, 1, a)
'''

'\ndef sum_sqrt(a):\n    return np.sqrt(np.sum(np.abs(a)**2, axis=-1))\n\ndef apply_norm_along_axis(a):\n    return np.apply_along_axis(np.linalg.norm, 1, a)\n'

In [4]:
# transition functions for MRP & MDP; M_x is 15x15 & M_e is 15x31 for M 30x30
m_x = np.random.randn(15,15)
m_e = np.random.randn(15,15)
print('m_x: \n', m_x)
print('m_e: \n', m_e)

m_x: 
 [[-0.19980464  0.61460971 -0.45196971  0.75581167 -1.20171605  0.00672903
   0.45975634  0.89457626  0.71817953 -0.34660912 -0.32183746  1.38967022
   0.82310815  0.1207746  -0.40816803]
 [-1.10917495 -0.33628095  0.27467286  1.04742835 -0.21070254 -0.94300722
   0.03290482 -0.39550715 -1.27413999  0.4075431   0.72290992  0.52010763
  -1.34282918 -0.23713391 -1.26415575]
 [-0.04847917 -0.28284972 -0.23142608  0.00734481  0.13320666 -0.74726323
  -1.02187831  0.19371526  0.5365637  -0.15997272  0.08980079  0.87552893
   0.22054629 -0.28268772 -0.8599804 ]
 [ 0.96280222 -0.18158858 -2.04114334  0.14835314  0.12842795  0.46120091
   0.40407166  0.30227455 -0.40884389 -0.14121077 -0.41893461  0.04790342
  -0.34201052 -0.28968019 -0.48976432]
 [-0.77271337 -0.27485742  0.57828462 -1.05660614  2.18928176  0.2614925
  -0.35108909  1.22945868 -0.21841512  1.16023069  0.48174255 -0.97862775
   0.0152969   1.47962361  0.95035897]
 [-0.34564359  0.56996016  0.2879895  -1.48523143 -0.081378

In [5]:
# normalize transition functions M_x & M_e using L2 norm
# np.linalg.norm(x, axis=1) is fastest way to compute the L2-norm
# L2-norm: each row's squared elements sum to 1 
m_x_norm = np.linalg.norm(m_x, axis=1)
m_e_norm = np.linalg.norm(m_e, axis=1)
print('m_x L2-norm: \n', m_x[0])
#print('m_x L2-norm: \n', m_e)

#print('sum: ', apply_norm_along_axis(m_x))

m_x L2-norm: 
 [-0.19980464  0.61460971 -0.45196971  0.75581167 -1.20171605  0.00672903
  0.45975634  0.89457626  0.71817953 -0.34660912 -0.32183746  1.38967022
  0.82310815  0.1207746  -0.40816803]


In [6]:
# verify l2-norm works
#norm_m_x = sum_sqrt(m_x)
#print(norm_m_x)

In [7]:
# state noise distributions: exo is N(0,0.09) & endo is N(0,0.04)
# N(mu, sigma^2) -> sigma * np.random.randn() + mu
sigma_x  = 0.3
sigma_e = 0.2
epsilon_x = sigma_x * np.random.randn()
epsilon_e = sigma_e * np.random.randn()
print(epsilon_x)
print(epsilon_e)

0.1367660024611725
-0.307804032921971


In [8]:
# State Transitions -- WIP 
# new_exo_state = Mx * previous_exo_state + epsilon_x
# new_endo_state = Me * previous_endo_state + epsilon_e

# update_x = mx * state_x + epsilon_x
# update_e = me * state_e + epsilon_e

In [9]:
# policy: action selection is done with 
#if exploration == "boltzmann":
    #Choose an action probabilistically, with weights relative to the Q-values.
    #Q_d, allQ = sess.run([q_net.Q_dist,q_net.Q_out],feed_dict={q_net.inputs:[s],q_net.Temp:e,q_net.keep_per:1.0})
    #a = np.random.choice(Q_d[0],p=Q_d[0])
    #a = np.argmax(Q_d[0] == a)

### Action Selection 

#### Softmax 

- actions are ranked and weighted according to value estimates 
- in this case we use a Boltzmann distribution 
- temperature value: 
        - high temperatures cause the actions to be nearly equiprobable
        - low temperatures cause the actions to have a greater difference in selection probability 

In [9]:
def action_selection(Qs, time_step, temperature):
    
    # loop through number of N -- what is N/? 
    
        # exp( Q(s_t, a) / temperature )
        num = math.exp(Qs[time_step,i]/temp)
        
        # sum_i of exp( Q(s_t, a_i) / temperature )  
        denom = sum(math.exp(val/temp) for val in Qs[time_step,:])
        
        # action_t ~ num/denom
        action = num / denom
        
        

Boltzmann Implementation Resource: https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-7-action-selection-strategies-for-exploration-d3a97b7cceaf

#Add this to network to compute Boltzmann probabilities.
Temp = tf.placeholder(shape=None,dtype=tf.float32)
Q_dist = slim.softmax(Q_out/Temp)

#Use this for action selection.
t = 0.5
Q_probs = sess.run(Q_dist,feed_dict={inputs:[state],Temp:t})
action_value = np.random.choice(Q_probs[0],p=Q_probs[0])
action = np.argmax(Q_probs[0] == action_value)

In [ ]:
# Add this to a network to compute Boltzmann probabilities 
temperature = 1.0
Q_dist = slim.softmax(Q_out/temperature)

# Use this for action selection 
temperature = 1.0 
Q_probs = sess.run(Q_dist, feed_dict={inputs: [state], temperature:t})
action_value = np.random.choice(Q_probs[0], p=Q_probs[0])
action = np.argmax(Q_probs[0] == action_value)